# Functional Analysis, Distributions & Weak Solutions

**Theme: The Geometry of Infinite Dimensions**

## 1. Introduction: The Cathedral of Analysis

In previously, we operated like 19th-century craftspeople: we used clever tricks, ansatzes, and heuristic manipulations to solve specific equations. We treated functions as smooth curves and derivatives as slopes. But we hit a wall. We encountered "functions" that weren't functions (the Dirac delta), series that didn't converge, and chaos that defied formulas.

To proceed, we must leave the comfortable world of explicit formulas and enter the rigorous cathedral of **Functional Analysis**. Here, we stop looking at the value of a function $f$ at a point $x$. Instead, we zoom out and view the function $f$ as a single **vector** in an infinite-dimensional space. The differential equation becomes a linear operator (a matrix of infinite size), and solving it becomes a problem of geometry: projections, angles, and orthogonality.


## 2. The Death of the Pointwise Function (Distributions)

Classical calculus breaks down when physics gets "sharp." A plucked string has a corner; a point charge has infinite density. To handle this, Laurent Schwartz (1950) revolutionized mathematics by inverting the definition of a function.

### 2.1 Test Functions and Functionals

Instead of asking "what is the value of $f$ at $x$?", we ask "what does $f$ do to a measurement?"

We define a space of **Test Functions**, $\mathcal{D}(\mathbb{R})$, consisting of infinitely smooth functions that vanish outside a compact set.

A **Distribution** $T$ is a continuous linear functional on this space. We denote its action as a pairing:

$$\langle T, \phi \rangle = T[\phi]$$

The most famous example is the **Dirac Delta**, which is nonsense as a function but perfectly well-defined as a distribution:

$$\langle \delta, \phi \rangle = \phi(0)$$


In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from scipy.integrate import trapezoid

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

# [The Mollifier. An interactive widget showing a sequence of smooth "bump" functions becoming narrower and taller, converging to the Dirac Delta. Users can see how ∫ φ_ε f dx stabilizes to f(0).]

class MollifierWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['epsilon'] = ParameterSlider(
            name='epsilon',
            value=1.0,
            min_val=0.01,
            max_val=2.0,
            step=0.01,
            description='Width $\\varepsilon$'
        )
        self.sliders['test_function'] = ParameterSlider(
            name='test_function',
            value=0,
            min_val=0,
            max_val=2,
            step=1,
            description='Test Function (0: Constant, 1: Linear, 2: Sine)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            eps = float(params['epsilon'])
            tf_type = int(params['test_function'])

            # Define x range
            x = np.linspace(-3, 3, 1000)

            # Standard mollifier (bump function)
            def mollifier(z):
                mask = np.abs(z) < 1
                result = np.zeros_like(z)
                result[mask] = np.exp(-1 / (1 - z[mask]**2))
                return result

            # Normalize to integrate to 1
            z = x / eps
            phi_unscaled = mollifier(z)
            # Normalize so that ∫ phi_ε dx = 1
            integral = trapezoid(phi_unscaled / eps, x)
            phi = phi_unscaled / (eps * integral) if integral > 0 else phi_unscaled / eps

            # Test functions
            if tf_type == 0:
                f = np.ones_like(x)  # Constant 1
                title_f = 'f(x) = 1'
            elif tf_type == 1:
                f = x  # Linear
                title_f = 'f(x) = x'
            else:
                f = np.sin(2 * np.pi * x)  # Sine
                title_f = 'f(x) = sin(2πx)'

            # Convolved value ≈ ∫ phi_ε(y) f(-y) dy ≈ f(0) as eps→0
            conv_value = trapezoid(phi * f, x)
            f_at_zero = f[len(f)//2]  # f(0) approximately

            title = f'Mollifier Approximation to Dirac Delta ($\\varepsilon={eps:.2f}$)<br>∫ $\\phi_\\varepsilon f$ dx ≈ {conv_value:.4f}, f(0)={f_at_zero:.4f} ({title_f})'

            # Create Plotly figure
            fig = self.plot_manager.create_plotly_figure(width=1000, height=600)

            # Plot mollifier
            fig.add_trace(
                go.Scatter(
                    x=x, y=phi, mode='lines',
                    name='$\\phi_\\varepsilon(x)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )

            # Plot test function (scaled for visibility)
            f_scaled = f / np.max(np.abs(f)) * np.max(phi) / 2 if np.max(np.abs(f)) > 0 else f
            fig.add_trace(
                go.Scatter(
                    x=x, y=f_scaled, mode='lines',
                    name=title_f,
                    line=dict(color=self.plot_manager.config.colors[3], width=2, dash='dash')
                )
            )

            fig.update_layout(
                title=title,
                xaxis_title='$x$',
                yaxis_title='Value',
                hovermode='x unified'
            )

            fig.show()

widget = MollifierWidget(title="Mollifier Convergence to Dirac Delta")
widget.display()


### 2.2 The Weak Derivative

How do we differentiate a function that isn't smooth? We steal the logic of Integration by Parts. If $u$ and $\phi$ are smooth, then:

$$\int u' \phi \, dx = -\int u \phi' \, dx$$

(The boundary terms vanish because $\phi$ has compact support).

For a distribution $T$, we *define* its derivative $T'$ to be the distribution that satisfies this identity for all test functions $\phi$:

$$\langle T', \phi \rangle = -\langle T, \phi' \rangle$$

This allows us to differentiate discontinuous functions, paving the way for shock waves and cracks in materials.


## 3. Measuring Energy: Sobolev Spaces

Distributions are too wild; they contain objects that are physically impossible to realize. We need to restrict ourselves to functions with finite energy.

### 3.1 The $L^2$ Hilbert Space

We define the space of square-integrable functions, $L^2(\Omega)$, equipped with an inner product:

$$\langle f, g \rangle_{L^2} = \int_\Omega f(x) g(x) \, dx$$

This structure allows us to measure "angles" between functions and define orthogonality, generalizing Euclidean geometry to infinite dimensions.

### 3.2 Sobolev Spaces ($H^s$)

To solve PDEs, we need control over derivatives. We define the **Sobolev Space** $H^1(\Omega)$ as the space of functions that are in $L^2$ and whose *weak derivatives* are also in $L^2$:

$$H^1(\Omega) = \{ u \in L^2(\Omega) : u' \in L^2(\Omega) \}$$

This is the natural setting for Quantum Mechanics and Thermodynamics, as the norm measures total energy (kinetic + potential):

$$\|u\|_{H^1}^2 = \int_\Omega (|u|^2 + |u'|^2) \, dx$$

**Sobolev Embedding Theorems** tell us when these "weak" functions are actually continuous, linking energy to regularity.


In [7]:
# [Sobolev Roughness. A visualizer contrasting a H^1 function (smooth) with an H^0 function (jagged, like a Brownian path, but with finite energy). Users toggle between spaces to see allowed "roughness."]

class SobolevRoughnessWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['space_type'] = ParameterSlider(
            name='space_type',
            value=0,
            min_val=0,
            max_val=1,
            step=1,
            description='Space (0: H⁰ (L²), 1: H¹)'
        )
        self.sliders['roughness'] = ParameterSlider(
            name='roughness',
            value=0.5,
            min_val=0.1,
            max_val=1.0,
            step=0.1,
            description='Roughness Level'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            is_h1 = params['space_type'] > 0.5
            rough = float(params['roughness'])

            # x range
            x = np.linspace(0, 1, 500)

            # Base smooth function
            smooth = np.sin(2 * np.pi * x) + 0.5 * np.sin(6 * np.pi * x)

            # Add roughness: for H^0, can have discontinuities; for H^1, continuous but derivative rough
            np.random.seed(42)
            if not is_h1:
                # H^0: square integrable, allow jumps
                noise = rough * (2 * np.random.rand(len(x)) - 1) * 0.1
                func = smooth + noise
                # Add a few jumps
                jump_points = np.random.choice(len(x), 3, replace=False)
                for jp in jump_points:
                    func[jp:] += rough * (2 * np.random.rand() - 1) * 0.2
                title = f'H⁰ (L²) Function - Roughness {rough:.1f}<br>(Can have discontinuities, finite L² norm)'
            else:
                # H^1: continuous, weak deriv in L²
                # Integrated white noise ~ Brownian-like but with bounded derivative
                noise = rough * np.cumsum(2 * np.random.rand(len(x)) - 1) / len(x) * 0.5
                func = smooth + noise
                title = f'H¹ Function - Roughness {rough:.1f}<br>(Continuous, derivative in L²)'

            # Normalize
            func = func / np.max(np.abs(func)) if np.max(np.abs(func)) > 0 else func

            # Compute derivative (for display)
            deriv = np.gradient(func, x)

            # Create figure
            fig = self.plot_manager.create_plotly_figure(width=1000, height=600)

            # Function trace
            fig.add_trace(
                go.Scatter(
                    x=x, y=func, mode='lines',
                    name='Function $u(x)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )

            # Derivative trace (scaled, on secondary y-axis)
            deriv_scaled = deriv / np.max(np.abs(deriv)) * np.max(np.abs(func)) if np.max(np.abs(deriv)) > 0 else deriv
            fig.add_trace(
                go.Scatter(
                    x=x, y=deriv_scaled, mode='lines',
                    name="Derivative $u'(x)$ (scaled)",
                    line=dict(color=self.plot_manager.config.colors[3], width=2, dash='dot')
                )
            )

            fig.update_layout(
                title=title,
                xaxis_title='$x$',
                yaxis_title='$u(x)$',
                hovermode='x unified'
            )

            fig.show()

widget = SobolevRoughnessWidget(title="Sobolev Spaces: Allowed Roughness")
widget.display()


## 4. The Spectral Theorem & Unbounded Operators

In Level 1, we assumed we could expand solutions into eigenfunctions ($u = \sum_n c_n \phi_n$). Level 2 rigorously justifies this via the **Spectral Theorem**.

### 4.1 Self-Adjoint Operators

The Laplacian $-\Delta$ is an **unbounded operator** on $L^2$. However, on a compact domain with boundary conditions, it behaves like a symmetric matrix. It allows for a spectral decomposition:

$$-\Delta u = \sum_n \lambda_n \langle u, \phi_n \rangle \phi_n$$

The eigenfunctions $\phi_n$ form a complete orthonormal basis for $L^2$. This is the mathematical justification for Fourier Series and the separation of variables.

### 4.2 Stone's Theorem & Semigroups

For time-dependent problems like the Schrödinger equation $i\hbar \partial_t \psi = H \psi$, we need to exponentiate operators. **Stone's Theorem** (1932) guarantees that if $H$ is self-adjoint, it generates a unitary group of time-evolution operators:

$$U(t) = e^{-itH/\hbar}$$

This establishes the one-to-one correspondence between physical observables (Hamiltonians) and time dynamics.


In [ ]:
# [The Spectral Drum. An interactive 2D domain where users draw a shape. The widget computes the Dirichlet eigenvalues and animates the eigenfunctions (vibrational modes) using a spectral solver.]

from scipy.sparse.linalg import eigs
from scipy.sparse import diags

class SpectralDrumWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['shape_type'] = ParameterSlider(
            name='shape_type',
            value=0,
            min_val=0,
            max_val=2,
            step=1,
            description='Shape (0: Square, 1: Circle, 2: L-Shape)'
        )
        self.sliders['mode'] = ParameterSlider(
            name='mode',
            value=1,
            min_val=1,
            max_val=10,
            step=1,
            description='Eigenmode Number'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            shape_type = int(params['shape_type'])
            mode_num = int(params['mode']) - 1  # 0-indexed

            # Grid setup
            n = 50  # Grid size
            h = 1.0 / (n - 1)
            x = np.linspace(0, 1, n)
            y = np.linspace(0, 1, n)
            X, Y = np.meshgrid(x, y)

            # Define domain mask
            if shape_type == 0:  # Square
                mask = np.ones((n, n), dtype=bool)
                title = 'Square Domain'
            elif shape_type == 1:  # Circle
                mask = (X - 0.5)**2 + (Y - 0.5)**2 <= 0.25
                title = 'Circular Domain'
            else:  # L-Shape: square minus upper-right quadrant
                mask = ~((X > 0.5) & (Y > 0.5))
                title = 'L-Shaped Domain'

            # Map grid indices to vector indices for interior points
            interior = np.where(mask.flatten())[0]
            N_int = len(interior)
            idx_map = {grid_idx: vec_idx for vec_idx, grid_idx in enumerate(interior)}

            # Build Laplacian matrix (finite difference, 5-point stencil)
            L = np.zeros((N_int, N_int))
            for k in range(N_int):
                grid_idx = interior[k]
                i, j = divmod(grid_idx, n)
                L[k, k] = -4.0 / (h**2)

                # Add neighbors (Dirichlet BC: boundary points = 0, so only interior neighbors)
                for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < n and 0 <= nj < n:
                        neigh_grid_idx = ni * n + nj
                        if neigh_grid_idx in idx_map:
                            neigh_vec_idx = idx_map[neigh_grid_idx]
                            L[k, neigh_vec_idx] = 1.0 / (h**2)

            # Compute eigenvalues/vectors (smallest magnitude = smallest positive eigenvalues of -L)
            try:
                vals, vecs = eigs(-L, k=min(10, N_int-1), which='SM')
                # Sort by real part
                idx = np.argsort(np.real(vals))
                vals = vals[idx]
                vecs = vecs[:, idx]

                if mode_num >= len(vals):
                    mode_num = len(vals) - 1

                # Reconstruct eigenfunction on grid
                eigfunc = np.zeros((n, n))
                eigfunc_flat = eigfunc.flatten()
                eigfunc_flat[interior] = np.real(vecs[:, mode_num])

                # Reshape back to grid
                eigfunc = eigfunc_flat.reshape((n, n))

                # Normalize
                max_val = np.max(np.abs(eigfunc_flat[interior]))
                if max_val > 1e-10:
                    eigfunc = eigfunc / max_val

                # Set values outside domain to NaN for cleaner visualization
                eigfunc[~mask] = np.nan

                # Plot with surface
                fig = self.plot_manager.create_plotly_figure(width=900, height=700)

                fig.add_trace(
                    go.Surface(
                        z=eigfunc, x=X, y=Y,
                        colorscale='RdBu',
                        colorbar=dict(title='$u(x,y)$'),
                        showscale=True
                    )
                )

                eigenval = np.real(vals[mode_num])
                fig.update_layout(
                    title=f'{title} - Eigenmode {mode_num+1}, $\\lambda \\approx {eigenval:.2f}$',
                    scene=dict(
                        xaxis_title='$x$',
                        yaxis_title='$y$',
                        zaxis_title='$u(x,y)$',
                        aspectmode='cube'
                    )
                )

                fig.show()
            except Exception as e:
                print(f"Error computing eigenvalues: {e}")

widget = SpectralDrumWidget(title="Spectral Drum: Dirichlet Eigenmodes")
widget.display()


## 5. Variational Methods (Lax-Milgram)

We now reach the most powerful tool for existence proofs: minimizing energy.

### 5.1 The Weak Formulation

Consider the Poisson equation $-\Delta u = f$. Multiply by a test function $v$ and integrate by parts:

$$\int_\Omega \nabla u \cdot \nabla v \, dx = \int_\Omega f v \, dx$$

This is the **Weak Formulation**: Find $u \in H^1_0(\Omega)$ such that $a(u, v) = \ell(v)$ for all $v \in H^1_0(\Omega)$, where $a$ is a bilinear form representing internal energy.

### 5.2 The Lax-Milgram Theorem

The **Lax-Milgram Theorem** (1954) is the "Fundamental Theorem of Linear PDEs." It states that if the bilinear form $a$ is:

1. **Bounded:** $|a(u, v)| \leq C \|u\| \|v\|$

2. **Coercive:** $a(u, u) \geq \alpha \|u\|^2$ (System has positive energy)

Then a unique solution exists. This replaces specific integration tricks with a universal geometric argument.


In [ ]:
# [The Energy Landscape. A 3D surface plot representing the energy functional J(u). The solution u is visualized as the unique ball sitting at the bottom of this infinite-dimensional paraboloid.]

class EnergyLandscapeWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['view_angle'] = ParameterSlider(
            name='view_angle',
            value=30,
            min_val=0,
            max_val=360,
            step=10,
            description='Azimuth Angle'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            azim = float(params['view_angle'])

            # Parameterize "function space" with two coords a,b; u = a sin(πx) + b sin(2πx)
            # Energy J(u) = ∫ (1/2 |u'|^2 - f u) dx, simplified to quadratic form
            a = np.linspace(-2, 2, 50)
            b = np.linspace(-2, 2, 50)
            A, B = np.meshgrid(a, b)

            # Quadratic energy: J = 1/2 (π² a² + 4π² b²) - (f1 a + f2 b)
            # Assume f1=1, f2=0.5 for visualization
            J = 0.5 * (np.pi**2 * A**2 + 4 * np.pi**2 * B**2) - (1.0 * A + 0.5 * B)

            # Minimum at grad J = 0: π² a - 1 = 0 => a = 1/π², 8π² b - 0.5 = 0 => b = 0.5/(8π²)
            min_a = 1.0 / np.pi**2
            min_b = 0.5 / (8 * np.pi**2)
            min_J = 0.5 * (np.pi**2 * min_a**2 + 4 * np.pi**2 * min_b**2) - (1.0 * min_a + 0.5 * min_b)

            fig = self.plot_manager.create_plotly_figure(width=900, height=700)

            # Energy surface
            fig.add_trace(
                go.Surface(
                    z=J, x=A, y=B,
                    colorscale='Viridis',
                    colorbar=dict(title='$J(u)$')
                )
            )

            # Add minimum point as a marker
            fig.add_trace(
                go.Scatter3d(
                    x=[min_a], y=[min_b], z=[min_J],
                    mode='markers',
                    marker=dict(
                        size=15,
                        color='red'
                    ),
                    name='Solution $u^*$'
                )
            )

            # Convert azimuth to radians for camera position
            azim_rad = np.deg2rad(azim)
            fig.update_layout(
                title='Energy Functional $J(u)$ in Projected Space<br>Minimum at Solution $u^*$ (Red Marker)',
                scene=dict(
                    xaxis_title='Coefficient $a$ ($\\sin(\\pi x)$)',
                    yaxis_title='Coefficient $b$ ($\\sin(2\\pi x)$)',
                    zaxis_title='$J(u)$',
                    camera=dict(
                        eye=dict(
                            x=1.5 * np.cos(azim_rad),
                            y=1.5 * np.sin(azim_rad),
                            z=0.5
                        )
                    )
                )
            )

            fig.show()

widget = EnergyLandscapeWidget(title="Energy Landscape for Variational Problems")
widget.display()


## 6. From Theory to Computation: Galerkin Methods

How do we solve infinite-dimensional problems on a computer? We project them.

### 6.1 Galerkin Projection

We choose a finite-dimensional subspace $V_h \subset H^1_0$ (e.g., piecewise polynomials). We seek an approximate solution $u_h \in V_h$ such that:

$$a(u_h, v_h) = \ell(v_h) \quad \forall v_h \in V_h$$

This turns the PDE into a matrix equation $Ku = F$.

### 6.2 Galerkin Orthogonality

The error $e = u - u_h$ is physically significant. The method satisfies **Galerkin Orthogonality**:

$$a(e, v_h) = 0 \quad \forall v_h \in V_h$$

Geometrically, this means the approximation $u_h$ is the *orthogonal projection* of the true solution onto the computational subspace with respect to the energy inner product. This is the foundation of the **Finite Element Method (FEM)**.


In [5]:
# [Galerkin Projection. A 2D visualization showing a "True" function curve and its "Finite Element" approximation. Users can increase the number of elements (dimension of V_h) and watch the approximation converge, visualizing the error orthogonality.]

from scipy.interpolate import interp1d
from scipy.integrate import trapezoid

class GalerkinProjectionWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['num_elements'] = ParameterSlider(
            name='num_elements',
            value=5,
            min_val=2,
            max_val=20,
            step=1,
            description='Number of Elements (dim $V_h$)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            n_elem = int(params['num_elements'])

            # Domain [0,1]
            x_fine = np.linspace(0, 1, 1000)

            # True function, e.g., u(x) = sin(2πx) + 0.5 sin(6πx)
            u_true = np.sin(2 * np.pi * x_fine) + 0.5 * np.sin(6 * np.pi * x_fine)

            # Finite element space: piecewise linear on n_elem elements
            nodes = np.linspace(0, 1, n_elem + 1)
            x_coarse = nodes

            # Project u_true onto V_h: interpolate for simplicity (close to projection for dense grid)
            u_h_coarse = np.sin(2 * np.pi * x_coarse) + 0.5 * np.sin(6 * np.pi * x_coarse)

            # Interpolate to fine grid
            interp = interp1d(x_coarse, u_h_coarse, kind='linear', fill_value='extrapolate')
            u_h_fine = interp(x_fine)

            # Error
            error = u_true - u_h_fine

            # L² error norm
            l2_error = np.sqrt(trapezoid(error**2, x_fine))

            title = f'Galerkin Approximation with {n_elem} Elements<br>Error $L^2$ Norm: {l2_error:.4f}'

            fig = self.plot_manager.create_plotly_figure(width=1000, height=600)

            # True solution
            fig.add_trace(
                go.Scatter(
                    x=x_fine, y=u_true, mode='lines',
                    name='True $u(x)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )

            # Approximation
            fig.add_trace(
                go.Scatter(
                    x=x_fine, y=u_h_fine, mode='lines',
                    name='Approximation $u_h(x)$',
                    line=dict(color=self.plot_manager.config.colors[3], width=2, dash='dash')
                )
            )

            # Error (scaled for visibility on secondary axis)
            error_scaled = error / np.max(np.abs(error)) * np.max(np.abs(u_true)) * 0.5 if np.max(np.abs(error)) > 0 else error
            fig.add_trace(
                go.Scatter(
                    x=x_fine, y=error_scaled, mode='lines',
                    name='Error $e(x)$ (scaled)',
                    line=dict(color=self.plot_manager.config.colors[1], width=2, dash='dot')
                )
            )

            fig.update_layout(
                title=title,
                xaxis_title='$x$',
                yaxis_title='$u(x)$',
                hovermode='x unified'
            )

            fig.show()

widget = GalerkinProjectionWidget(title="Galerkin Projection and Convergence")
widget.display()


## 7. Solvability & Fredholm Theory

Finally, we address the question: "Is the solution unique?"

For many operators, the answer is "No." **Fredholm Theory** classifies linear operators by their **Index**:

$$\text{ind}(L) = \dim \ker(L) - \dim \text{coker}(L)$$

This integer describes the mismatch between the number of independent solutions and the constraints on the data. It is the first hint of *topology* entering analysis—an integer invariant that remains constant under perturbation, foreshadowing the geometric levels to come.


---

### References

* **Céa, J.** (1964). *Approximation variationnelle des problèmes aux limites*.

* **Fredholm, I.** (1903). *Sur une classe d'équations fonctionnelles*.

* **Galerkin, B. G.** (1915). *Series developments for some cases of equilibrium of plates and beams*.

* **Lax, P. D., & Milgram, A. N.** (1954). *Parabolic equations*.

* **Schwartz, L.** (1950). *Théorie des distributions*.

* **Sobolev, S. L.** (1938). *On a theorem of functional analysis*.

* **Stone, M. H.** (1932). *Linear transformations in Hilbert space*.

* **von Neumann, J.** (1927). *Mathematische Begründung der Quantenmechanik*.
